In [1]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta

# Setup Faker
fake = Faker()

# PostgreSQL connection details
conn = psycopg2.connect(
    dbname="sw_warehouse",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Create table
cur.execute("""
    DROP TABLE IF EXISTS monsters;
    CREATE TABLE monsters (
        monster_id SERIAL PRIMARY KEY,
        name VARCHAR(100),
        scare_level INT,
        department VARCHAR(50)
    );
""")

# Insert random data
departments = ['Scare Floor A', 'Scare Floor B', 'R&D', 'Administration']

for _ in range(10):
    name = fake.first_name()
    scare_level = random.randint(1, 10)
    department = random.choice(departments)
    
    cur.execute(
        "INSERT INTO monsters (name, scare_level, department) VALUES (%s, %s, %s)",
        (name, scare_level, department)
    )

# Commit and close
conn.commit()

print("Table 'monsters' created and populated with random data!")


Table 'monsters' created and populated with random data!
Table 'monsters' created and populated with random data!


In [2]:
# Create scarers table
cur.execute("""
    CREATE TABLE IF NOT EXISTS scarers (
        scarer_id SERIAL PRIMARY KEY,
        monster_id INT,
        assigned_floor VARCHAR(50),
        active BOOLEAN,
        FOREIGN KEY (monster_id) REFERENCES monsters(monster_id)
    );
""")

# Fetch monster IDs from the monsters table
cur.execute("SELECT monster_id FROM monsters;")
monster_ids = [row[0] for row in cur.fetchall()]

# Randomly select a subset to be scarers
num_scarers = random.randint(int(len(monster_ids) * 0.4), int(len(monster_ids) * 0.8))
scarer_monsters = random.sample(monster_ids, num_scarers)

# Insert randomized scarers
for monster_id in scarer_monsters:
    assigned_floor = random.choice(['Scare Floor A', 'Scare Floor B', 'Scare Floor C', 'Overflow'])
    active = random.choice([True, False, True, True])  # Higher chance of active
    cur.execute("""
        INSERT INTO scarers (monster_id, assigned_floor, active)
        VALUES (%s, %s, %s);
    """, (monster_id, assigned_floor, active))

conn.commit()

print(f"{len(scarer_monsters)} scarers inserted successfully!")

7 scarers inserted successfully!
7 scarers inserted successfully!


In [3]:
try:
    # Create the doors table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS doors (
        door_id SERIAL PRIMARY KEY,
        door_serial_number VARCHAR(50) UNIQUE NOT NULL,
        door_history_id INT NOT NULL,
        warehouse_location_id INT NOT NULL,
        timezone_location_id INT NOT NULL,
        status VARCHAR(20) NOT NULL,
        last_service_date DATE,
        times_used INT DEFAULT 0,
        total_energy_output NUMERIC(10, 2) DEFAULT 0.00,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    cur.execute(create_table_query)
    conn.commit()
    print("Table 'doors' created successfully!")

except Exception as e:
    conn.rollback()
    print(f"Error creating table: {e}")

# Generate random door data
def generate_door_data(num_records=50):
    door_data = []
    for _ in range(num_records):
        door_serial = fake.unique.bothify(text='DOOR-####-????')
        door_history_id = random.randint(1, 10)
        warehouse_location_id = random.randint(1, 10)
        timezone_location_id = random.randint(1, 10)
        status = random.choice(['available', 'in-use', 'maintenance', 'retired'])
        last_service_date = fake.date_between(start_date='-2y', end_date='today')
        times_used = random.randint(0, 500)
        total_energy_output = round(random.uniform(0, 10000), 2)
        created_at = fake.date_time_between(start_date='-3y', end_date='now')

        door_data.append((
            door_serial, door_history_id, warehouse_location_id, timezone_location_id,
            status, last_service_date, times_used, total_energy_output, created_at
        ))
    return door_data

# Insert data safely
def insert_doors(data):
    try:
        insert_query = """
            INSERT INTO doors (
                door_serial_number, door_history_id, warehouse_location_id, timezone_location_id,
                status, last_service_date, times_used, total_energy_output, created_at
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        cur.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} doors inserted successfully!")
    except Exception as e:
        conn.rollback()
        print(f"Error inserting data: {e}")

# Run the full process
door_records = generate_door_data(50)
insert_doors(door_records)

# Cleanup
cur.close()
conn.close()

Table 'doors' created successfully!
50 doors inserted successfully!
Table 'doors' created successfully!
50 doors inserted successfully!
